# Data Pre-processing


In [25]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import utils

In [26]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [27]:
data = pd.read_csv('../Data/train_data.csv', index_col='Claim Identifier')


C:\Users\isa\AppData\Local\Temp\ipykernel_17728\1307583070.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../Data/train_data.csv', index_col='Claim Identifier')


_____

# Empty rows
In the initial analysis we saw that the only features that has no missing values is `_Assembly Date_`. So let's check how many rows only have that feature filled and the rest of columns empty

In [28]:
data.drop(columns=['Assembly Date']).isna().all(axis=1).sum()


19445

In [29]:
df = data[~(data.drop(columns=['Assembly Date']).isna().all(axis=1) & data['Assembly Date'].notna())]

In [30]:
df.drop(columns=['Assembly Date']).isna().all(axis=1).sum()


0

Here 19445 empty rows were dropped

_____

# Overlook of the features
This notebook will include pre processing of the following variables:
- Carrier Name 
- Attorney Representative
- OIICS Nature of Injury Description
- Industry Code
- Medical Fee Region
- C-2 Date
- First Hearing Date
- WCIO Cause of Injury Code
- WCIO Part of Body Code
- WCIO Cause of Injury Description
- WCIO Nature of Injury Description

Since we dropped some rows in the previous section we should check again for the updated number of missing values for the features we are interest in

In [31]:
target_features = ['Carrier Name', 'Attorney/Representative', 'OIICS Nature of Injury Description', 
                   'Industry Code', 'Medical Fee Region', 'C-2 Date', 'First Hearing Date','WCIO Cause of Injury Code',
                   'WCIO Part Of Body Code','WCIO Cause of Injury Description','WCIO Nature of Injury Description']

In [32]:
df[target_features].isnull().sum()/df.shape[0]*100

Carrier Name                            0.000000
Attorney/Representative                 0.000000
OIICS Nature of Injury Description    100.000000
Industry Code                           1.734765
Medical Fee Region                      0.000000
C-2 Date                                2.536470
First Hearing Date                     73.729761
WCIO Cause of Injury Code               2.724615
WCIO Part Of Body Code                  2.975823
WCIO Cause of Injury Description        2.724615
WCIO Nature of Injury Description       2.727577
dtype: float64

The features _`Carrier Name`_,  _`Attorney/Representative`_ and _`Medical Fee Region`_ no longer have missing values. Let's look at the others

### OIICS Nature of Injury Description
Between all the complains this feature is never filled, so we can drop it

In [33]:
# drop OIICS Nature of Injury Description
df = df.drop(columns=['OIICS Nature of Injury Description'])

### First Hearing Date
This feature also has a high percentage of missing values. According to the provided information for this project, 'A blank date means the claim has not yet had a hearing held'. So we should create a label for the ones that hadn't had a hearing yet, we will simply call it 'N/A'

In [34]:
# transform First Hearing Date into pd.datetime
df['First Hearing Date'] = pd.to_datetime(df['First Hearing Date'])

In [35]:
# fill NA with 'N/A'
df['First Hearing Date'] = df['First Hearing Date'].fillna('N/A')
df['First Hearing Date'].isna().sum()

0

To assist in the information that this feature can provide we can create another feature called _`Has First Hearing Date`_ - a binary value that indicates if the user has had it's first hearing date

In [36]:
# create Has First Hearing Date based on if is 'N/A' or not
df['Has First Hearing Date'] = df['First Hearing Date'].apply(lambda x: 0 if x == 'N/A' else 1)

In [37]:
df['Has First Hearing Date'].value_counts()

Has First Hearing Date
0    423228
1    150798
Name: count, dtype: int64

### C-2 Date
I think that there is relationship between this C-2 Date and some other date, ideally _`Assembly Date`_, but I dont know how to prove it yet

In [38]:
date_features = ['Accident Date', 'C-2 Date', 'First Hearing Date', 'Assembly Date']
date_df = df[date_features]

date_df[date_df['C-2 Date'].isna()].head(20)

,Accident Date,C-2 Date,First Hearing Date,Assembly Date
Claim Identifier,,,,
5393991,2019-12-05,NaN,N/A,2020-01-02
5394018,2019-12-11,NaN,2020-09-15 00:00:00,2020-01-02
5394153,2019-08-15,NaN,N/A,2020-01-02
5393971,2019-06-26,NaN,N/A,2020-01-02
5393773,2019-12-27,NaN,N/A,2020-01-02
5393917,2019-09-23,NaN,N/A,2020-01-02
5394102,2019-12-20,NaN,N/A,2020-01-02
5393964,2001-09-12,NaN,N/A,2020-01-02
5393769,2019-11-14,NaN,N/A,2020-01-02


### Codes and Descriptions
This section will analyse data related to codes and their descriptions

In [39]:
df_codes = df[[col for col in df.columns if 'Code' in col]]
df_codes.drop(columns=['Industry Code Description', 'Zip Code'], inplace=True)

df_description = df[[col for col in df.columns if 'Description' in col]]

C:\Users\isa\AppData\Local\Temp\ipykernel_17728\1033338758.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_codes.drop(columns=['Industry Code Description', 'Zip Code'], inplace=True)


In [40]:
# ensure that all codes are objects
df_codes = df_codes.astype('object')

df_codes.describe()

,Industry Code,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code
count,564068.0,558386.0,558369.0,556944.0
unique,24.0,77.0,56.0,57.0
top,62.0,56.0,52.0,42.0
freq,114339.0,46610.0,153373.0,51862.0


In [41]:
#check the order of the codes
df_codes.columns


Index(['Industry Code', 'WCIO Cause of Injury Code',
       'WCIO Nature of Injury Code', 'WCIO Part Of Body Code'],
      dtype='object')

In [42]:
#check the order of the descriptions
df_description.columns

Index(['Industry Code Description', 'WCIO Cause of Injury Description',
       'WCIO Nature of Injury Description', 'WCIO Part Of Body Description'],
      dtype='object')

In [ ]:
def code_description_consistency(df, code_column, description_column):
    """
    Checks the one-to-one consistency between a code and description column in a DataFrame.
    
    Parameters:
    - df (pd.DataFrame): The DataFrame containing the data.
    - code_column (str): The name of the column representing codes.
    - description_column (str): The name of the column representing descriptions.
    
    Returns:
    - tuple: A tuple with two boolean values:
        - codes_match_descriptions (bool): True if each code has a unique description.
        - descriptions_match_codes (bool): True if each description has a unique code.
    """
    # Remove rows with missing values in either the code or description columns
    df_non_null = df.dropna(subset=[code_column, description_column])

    # Check if each code maps to a unique description
    code_consistency = df_non_null.groupby(code_column)[description_column].nunique() == 1

    # Check if each description maps to a unique code
    description_consistency = df_non_null.groupby(description_column)[code_column].nunique() == 1

    # Check overall consistency
    codes_match_descriptions = code_consistency.all()
    descriptions_match_codes = description_consistency.all()

    return codes_match_descriptions, descriptions_match_codes

In [43]:
#since they match we can iterate between both dataframes and use code_description_consistency()

for code, description in zip(df_codes.columns, df_description.columns):
    #print the result of the function
    print(code_description_consistency(df_codes[code], df_description[description]))

NameError: name 'code_description_consistency' is not defined